## Tool Selection - PoC
Here, we will use the LLM to select the best tool out of a filtered set of (at most) 20 tables.

Let's get started.

In [1]:
from pathlib import Path
import os

root = Path().absolute().parents[1]
os.chdir(str(root))

In [3]:
from src.helpers.hybrid_retrieval import HybridRetrieval

In [4]:
retriever = HybridRetrieval(top_k_stage_1=200, top_k_stage_2=20)

In [ ]:
# 4) Query
question = "does the fish purr like a cat?"

question = "What's the electricity generation mix in Ireland? for renewable and non-renewable energy?"
# question = "What is the share of renewable energy in Ireland?"
# question = "What's the breakup between renewable and non-renewable energy production in Ireland?"
question = "Ireland rural vs urban population mix"
# question = "What are ireland's top exports?"
# question = "What are Prodcom sales for skincare beauty and makeup products in 2023?"
# question = "Give me a breakup of Ireland's share of transportation sector."
# question = "Sold productions - quantity (kg) for beauty, makeup and skincare preparations in Ireland in 2023?"
question = "beauty makeup & skincare production in prodcom data for ireland in 2023"
# question = "PRODCOM production quantity."
# question = "What's the mining and quarrying production in Ireland?"
# question = "nano cellulose production in ireland"
# question = "give me pharmaceuticals production in ireland"
# question = "what are the different types of pharmaceuticals products produced in ireland"


response = retriever.search(query=question)
response

I0000 00:00:1755784664.165868 1316053 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


,id,stage_1_score,stage_2_score
0,CPM13,8.789187,0.460212
1,PCA23,4.585402,0.328005
2,IAIP13,7.194671,0.396397
3,CPM16,7.114656,0.444113
4,CPM18,7.114656,0.436573
...,...,...,...
71,NSA88,4.598532,0.422409
72,NSA97,4.598532,0.429908
73,MIP26,4.596580,0.407813
74,MIP10,4.577620,0.408266
